In [2]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top= False)

In [3]:
# Freeze base model
base_model.trainable = False

In [4]:
# Create inputs with correct shape
inputs = keras.Input(shape = (224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(rotation_range=10,
    zoom_range=0.1,  
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True, )
datagen_valid = ImageDataGenerator(samplewise_center=True)

In [8]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "data/fruits/train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "data/fruits/valid",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 1180 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


In [9]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=15)

Epoch 1/15
36/36 [==============================] - 162s 4s/step - loss: 2.8363 - accuracy: 0.4500 - val_loss: 1.3474 - val_accuracy: 0.6261
Epoch 2/15
36/36 [==============================] - 173s 5s/step - loss: 0.8747 - accuracy: 0.7373 - val_loss: 0.8991 - val_accuracy: 0.7508
Epoch 3/15
36/36 [==============================] - 183s 5s/step - loss: 0.4645 - accuracy: 0.8593 - val_loss: 0.5911 - val_accuracy: 0.8541
Epoch 4/15
36/36 [==============================] - 197s 5s/step - loss: 0.3028 - accuracy: 0.9025 - val_loss: 0.5201 - val_accuracy: 0.8632
Epoch 5/15
22/36 [================>.............] - ETA: 1:07 - loss: 0.2073 - accuracy: 0.9443

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)